In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
! pip install tensorflow_addons
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.12.0
Running on TPU  ['10.49.77.178:8470']


## Load the Data for training

In [ ]:
def parse(serialized,signal_shape=(1024,12),label_shape=(1,)):

    features = {'signal': tf.io.FixedLenFeature([], tf.string), 'label': tf.io.FixedLenFeature([], tf.int64)}
    parsed_example = tf.io.parse_single_example(serialized=serialized, features=features)

    signal = parsed_example['signal']
    label = parsed_example['label']

    signal = tf.io.decode_raw(signal, tf.float64)
    signal = tf.reshape(signal, shape=signal_shape)
    signal = tf.transpose(signal)
    signal = signal*(10**5)

    # label = tf.io.decode_raw(label, tf.int64)
    label = tf.reshape(label, shape=label_shape)

    return signal, label

In [ ]:
AUTOTUNE=tf.data.AUTOTUNE
train_files = tf.io.matching_files('gs://tinyml/train_fyp/*train.tfrec')
train_files = tf.random.shuffle(train_files)
shards = tf.data.Dataset.from_tensor_slices(train_files)
train_dataset = shards.interleave(lambda x: tf.data.TFRecordDataset(x), num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.map(parse, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(128)
train_dataset = train_dataset.prefetch(AUTOTUNE)

AUTOTUNE=tf.data.AUTOTUNE
test_files = tf.io.matching_files('gs://tinyml/test_fyp/*test.tfrec')
shards = tf.data.Dataset.from_tensor_slices(test_files)
test_dataset = shards.interleave(lambda x: tf.data.TFRecordDataset(x), num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(parse, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(128)
test_dataset = test_dataset.prefetch(AUTOTUNE)

In [ ]:
# for i in train_dataset:
#   print(i[0][0].shape)
#   print(i[1][0][0])
#   plt.plot(i[0][0][:,0])
#   break

pos = 0
tot = 0
for i in train_dataset:
  # print(i[1].shape)
  pos+=sum(i[1])
  tot+=i[1].shape[0]
  # break

pos = pos.numpy()[0]
print(pos, tot)

Exception ignored in: <function Executor.__del__ at 0x7f660c5288b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/executor.py", line 46, in __del__
    self.wait()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/executor.py", line 65, in wait
    pywrap_tfe.TFE_ExecutorWaitForAllPendingNodes(self._handle)
tensorflow.python.framework.errors_impl.OutOfRangeError: End of sequence


5325 61392


## Develope a large model

In [ ]:
def block_type1(n_filters1,n_filters2, kernel, pad, x):
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(x)
    block1=layers.BatchNormalization()(block1)
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block1)
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block1)
    block1=layers.BatchNormalization()(block1)
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block1)
    block1=layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(block1)
    block1=layers.BatchNormalization()(block1)

    block1_o=layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(x)
    block1_o=layers.BatchNormalization()(block1_o)

    added1=layers.add([block1,block1_o])
    return layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(added1)

def block_type2(n_filters1,n_filters2, kernel, pad, x):
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(x)
    block=layers.BatchNormalization()(block)
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block)
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block)
    block=layers.BatchNormalization()(block)
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block)
    block=layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(block)
    block=layers.BatchNormalization()(block)

    added=layers.add([x,block])
    return layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(added)

In [ ]:
def create_model():
  Input=keras.Input(shape=(1024,12),name="Input_signal")
  x=layers.Conv1D(32,3,padding="same",activation='swish')(Input)
  x=layers.BatchNormalization()(x)
  x=layers.Conv1D(32,3,padding="same",activation='swish')(x)
  x=layers.MaxPool1D(2)(x)

  x1=block_type1(32,128,3,"same",x)
  x2=block_type2(32,128,3,"same",x1)
  x3=block_type2(32,128,3,"same",x2)
  x3=layers.AveragePooling1D(4)(x3)
  x4=block_type1(64,256,7,"same",x3)
  x5=block_type2(64,256,7,"same",x4)
  x6=block_type2(64,256,7,"same",x5)
  x7=block_type2(64,256,7,"same",x6)
  # x7=layers.AveragePooling1D(4)(x7)
  # x8=block_type1(128,512,9,"same",x7)
  # x9=block_type2(128,512,9,"same",x8)
  # x10=block_type2(128,512,9,"same",x9)
  # x11=block_type2(128,512,9,"same",x10)
  # x12=block_type2(128,512,9,"same",x11)
  # x13=block_type2(128,512,9,"same",x12)
  # x13=layers.AveragePooling1D(4)(x13)
  # x14=block_type1(256,1024,11,"same",x13)
  # x15=block_type2(256,1024,11,"same",x14)
  # x16=block_type2(256,1024,11,"same",x15)
  x16=layers.GlobalAveragePooling1D()(x7)
  x16=layers.Dense(1)(x16)

  model=keras.Model(Input,x16)
  F1 = tfa.metrics.FBetaScore(num_classes=1)
  AUROC = tf.keras.metrics.AUC(curve='ROC', name = 'AUROC')
  AUPRC = tf.keras.metrics.AUC(curve='PR', name = 'AUPRC')
  model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy', F1, AUROC, AUPRC])
  keras.utils.plot_model(model,'model.png',show_layer_activations=True,show_shapes=True)
  model.summary()
  return model

with tpu_strategy.scope():
  model = create_model()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_signal (InputLayer)      [(None, 1024, 12)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 1024, 32)     1184        ['Input_signal[0][0]']           
                                                                                                  
 batch_normalization (BatchNorm  (None, 1024, 32)    128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv1d_1 (Conv1D)              (None, 1024, 32)     3104        ['batch_normalization[0][0]']

In [ ]:
def create_model():
    i = keras.Input((1024,12), name='signal')
    x = layers.Conv1D(filters=72, kernel_size=15, activation='swish', padding='same')(i)
    a = layers.Conv1D(filters=72, kernel_size=1, padding='same')(i)
    a = layers.Add()([a,x])
    a = layers.Activation('swish')(a)
    x = layers.SpatialDropout1D(0.2)(a)
    b = layers.AveragePooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=144, kernel_size=3, activation='swish', padding='same')(b)
    a = layers.Conv1D(filters=144, kernel_size=1, padding='same')(b)
    a = layers.Add()([a,x])
    a = layers.Activation('swish')(a)
    x = layers.SpatialDropout1D(0.2)(a)
    b = layers.AveragePooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=288, kernel_size=5, activation='swish', padding='same')(b)
    a = layers.Conv1D(filters=288, kernel_size=1, padding='same')(b)
    a = layers.Add()([a,x])
    a = layers.Activation('swish')(a)
    x = layers.SpatialDropout1D(0.2)(a)
    b = layers.AveragePooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=576, kernel_size=7, activation='swish', padding='same')(b)
    a = layers.Conv1D(filters=576, kernel_size=1, padding='same')(b)
    a = layers.Add()([a,x])
    a = layers.Activation('swish')(a)
    b = layers.SpatialDropout1D(0.2)(a)
    x = layers.GlobalAveragePooling1D()(b)

    # j = keras.Input((2000,2*2), name='fft')
    # y = layers.Conv1D(filters=72, kernel_size=3, activation='swish', padding='same')(j)
    # y = layers.SpatialDropout1D(0.1)(y)
    # y = layers.AveragePooling1D(pool_size=2)(y)
    # y = layers.Conv1D(filters=144, kernel_size=5, activation='swish', padding='same')(y)
    # y = layers.SpatialDropout1D(0.1)(y)
    # y = layers.AveragePooling1D(pool_size=2)(y)
    # y = layers.Conv1D(filters=288, kernel_size=7, activation='swish', padding='same')(y)
    # y = layers.SpatialDropout1D(0.1)(y)
    # y = layers.AveragePooling1D(pool_size=2)(y)
    # y = layers.Conv1D(filters=576, kernel_size=9, activation='swish', padding='same')(y)
    # y = layers.SpatialDropout1D(0.1)(y)
    # y = layers.AveragePooling1D(pool_size=2)(y)
    # y = layers.Conv1D(filters=1152, kernel_size=11, activation='swish', padding='same')(y)
    # y = layers.SpatialDropout1D(0.1)(y)
    # y = layers.GlobalAveragePooling1D()(y)

    # f = layers.Concatenate()([x,y])
    # f = layers.Activation('swish')(f)

    f = layers.Dense(576, activation='swish')(x)
    f = layers.Dropout(0.5)(f)
    f = layers.Dense(1,activation = 'sigmoid')(f)

    F1 = tfa.metrics.FBetaScore(num_classes=1)
    AUROC = tf.keras.metrics.AUC(curve='ROC', name = 'AUROC')
    AUPRC = tf.keras.metrics.AUC(curve='PR', name = 'AUPRC')
    model = keras.Model(i,f)
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy', AUROC, AUPRC])
    tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_dtype=False,show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96,layer_range=None, show_layer_activations=False)
    return model

with tpu_strategy.scope():
  model = create_model()


In [ ]:
weight_for_0 = (1 / (tot-pos)) * (tot / 2.0)
weight_for_1 = (1 / pos) * (tot / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(train_dataset, validation_data = test_dataset, epochs=10, class_weight=class_weight)

Epoch 1/10


ValueError: ignored

## Small model

In [ ]:

# import tensorflow_addons as tfa
# from sklearn.model_selection import train_test_split

model=keras.Sequential(name="BrainNeoCare_CNN")
model.add(keras.layers.Conv2D(filters=32, kernel_size=(1,3), activation='relu', input_shape=(12,1024,1),name="Input_conv2D"))
model.add(keras.layers.MaxPooling2D((1,2),name="Maxpooling2D_1"))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(1,3), activation='relu',name="Conv2D_2"))
model.add(keras.layers.MaxPooling2D((1,2),name="Maxpooling2D_2"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(1,3), activation='relu',name="Conv2D_3"))
model.add(keras.layers.MaxPooling2D((1,2),name="Maxpooling2D_3"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(1,3), activation='relu',name="Conv2D_4"))
model.add(keras.layers.AveragePooling2D((1,2),name="AveragePooling2D_1"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu',name="2D_F_Con"))
model.add(keras.layers.AveragePooling2D((1,2),name="AveragePooling2D_2"))
model.add(keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu',name="2D_F_Con_2"))
model.add(keras.layers.MaxPooling2D((1,2),name="AveragePooling2D_3"))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu') )
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, name="binary_crossentropy",)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train,Y_train,epochs=10,batch_size=32,validation_split=0.2,verbose=1)

In [ ]:
x=np.load('/content/drive/MyDrive/FYP_ML/zenodo_data.npy', mmap_mode='r')
y=np.load('/content/drive/MyDrive/FYP_ML/zenodo_labels.npy', mmap_mode='r')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
y_train_positive_pos=np.where(y_train==1)
y_test_positive_pos=np.where(y_test==1)

x_train_positive=x_train[y_train_positive_pos]
x_test_positive=x_test[y_test_positive_pos]

y_train_negative_pos=np.where(y_train==0)
y_test_negative_pos=np.where(y_test==0)

x_train_negative=x_train[y_train_negative_pos][0:6000,...]
x_test_negative=x_test[y_test_negative_pos][0:2000,...]

new_x_train=np.concatenate((x_train_positive,x_train_negative),axis=0)
new_y_train=np.concatenate((np.ones((x_train_positive.shape[0],1)),np.zeros((x_train_negative.shape[0],1))),axis=0)

new_x_test=np.concatenate((x_test_positive,x_test_negative),axis=0)
new_y_test=np.concatenate((np.ones((x_test_positive.shape[0],1)),np.zeros((x_test_negative.shape[0],1))),axis=0)

np.save('/content/drive/MyDrive/FYP_ML/balanced_x_train.npy',new_x_train)
np.save('/content/drive/MyDrive/FYP_ML/balanced_y_train.npy',new_y_train)

np.save('/content/drive/MyDrive/FYP_ML/balanced_x_test.npy',new_x_test)
np.save('/content/drive/MyDrive/FYP_ML/balanced_y_test.npy',new_y_test)

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(signal, label):
  feature = {
      'signal': _bytes_feature(signal),
      'label':_int64_feature(label)
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
CT = 16
SIZE = len(x_train)//CT

for j in range(CT):
    print(); print('Writing TFRecord %i of %i...'%(j,CT))
    CT2 = min(SIZE,len(x_train)-j*SIZE)
    with tf.io.TFRecordWriter('%.2i-%i_train.tfrec'%(j,CT2)) as writer:
        for k in range(CT2):
            signal = x_train[SIZE*j+k].tobytes()
            label = y_train[SIZE*j+k]
            example = serialize_example(signal,label)
            writer.write(example)
            if k%100==0: print(k,', ',end='')
    !gsutil -m cp -r *train.tfrec gs://tinyml/train_fyp/
    !rm *train.tfrec


Writing TFRecord 0 of 16...
0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , Copying file://00-3837_train.tfrec [Content-Type=application/octet-stream]...
Copying file://04-3837_train.tfrec [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a c

In [ ]:
CT = 4
SIZE = len(x_test)//CT

for j in range(CT):
    print(); print('Writing TFRecord %i of %i...'%(j,CT))
    CT2 = min(SIZE,len(x_test)-j*SIZE)
    with tf.io.TFRecordWriter('%.2i-%i_test.tfrec'%(j,CT2)) as writer:
        for k in range(CT2):
            signal = x_test[SIZE*j+k].tobytes()
            label = y_test[SIZE*j+k]
            example = serialize_example(signal,label)
            writer.write(example)
            if k%100==0: print(k,', ',end='')
    !gsutil -m cp -r *test.tfrec gs://tinyml/test_fyp/
    !rm *test.tfrec


Writing TFRecord 0 of 4...
0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , Copying file://00-3837_test.tfrec [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil 